In [ ]:
#Let's start by importing the required libraries
!pip install strawberryfields
!pip install scipy
import numpy as np
import strawberryfields as sf
import matplotlib.pyplot as plt
from scipy.special import erfc

In [ ]:
#SECTION 1 STARTS HERE

In [ ]:
#Let's model a Poisson Point Process with a mean of 0.5
mean = 1
#Let's generate 100 samples from this Poisson Point Process
samples = np.random.poisson(mean, 1000000)
#Lets create a histogram of the samples
import matplotlib.pyplot as plt
plt.hist(samples,bins=100)

In [ ]:
#Let's plot the samples for a homodyne detection
#Let's define the mean and variance of the Gaussian
mu1 = 1
var1 = 1/2
#Let's generate 100 samples from this Gaussian
samples1 = np.random.normal(mu1, var1, 100000)
#Let's plot the samples
#plt.hist(samples1,1000)

#Let's plot samples for homodyne detection of another gaussian and compare the two
#Let's define the mean and variance of the Gaussian
mu2 = -1
var2 = 1/2
#Let's generate 100 samples from this Gaussian
samples2 = np.random.normal(mu2, var2, 100000)
#Let's plot the samples
#plt.hist(samples2,1000)

#Plot both the gaussians together
plt.hist(samples1,1000)
plt.hist(samples2,1000)
#Draw a line at x=0
plt.axvline(x=0, color='red')


In [ ]:
#Now let's model homodyne in Strawberry Fields

#This is the main function that we will use to generate samples
def BPSK_Sample_Homodyne(alpha):
    #Let's define the Strawberry Fields program, we will only use a single mode
    prog = sf.Program(1)
    #Let's define the Strawberry Fields engine, we will use the Gaussian backend
    eng = sf.Engine('gaussian')

    #The circuit will place a coherent state with complex amplitude alpha into mode 0, then measure homodyne
    with prog.context as q:
        sf.ops.Coherent(alpha)| q[0]
        sf.ops.MeasureHomodyne(0) | q[0]

    #Let's run the circuit
    result = eng.run(prog)
    #let's return the samples
    return result.samples[0]

In [ ]:
#Now let's run it many times
#print(BPSK_Sample_Homodyne(1)[0])

sample_count=10000

samples = np.zeros(sample_count)
for i in range(sample_count):
    samples[i]=BPSK_Sample_Homodyne(1)[0]

#Let's plot the samples
plt.hist(samples,1000)

In [ ]:
#Lets plot the coherent states in phase space
#Let's define the Strawberry Fields program, we will only use a single mode
prog = sf.Program(2)
#Let's define the Strawberry Fields engine, we will use the Gaussian backend
eng = sf.Engine('gaussian')

alpha=1

#The circuit will place a coherent state with complex amplitude alpha into mode 0, then measure homodyne
with prog.context as q:
    sf.ops.Coherent(alpha)| q[0]
    sf.ops.Coherent(-alpha) | q[1]

#Let's run the circuit
result = eng.run(prog)

#Let's visualize the state in mode 0
x = np.linspace(-5, 5, 100) #This creates a 1D array of 100 points between -5 and 5
y = result.state.wigner(0, x, x) #Returns pre-calculated Wigner function from the state of the quantum register
X, Y = np.meshgrid(x, x) #Plotting
plt.contourf(X, Y, y)
plt.show()

state = result.state
fig1 = plt.figure()
X = np.linspace(-5, 5, 100)
P = np.linspace(-5, 5, 100)
Z = state.wigner(0, X, P)
X, P = np.meshgrid(X, P)
ax = fig1.add_subplot(111, projection="3d")
fig1.set_size_inches(4.8, 5)
ax.plot_surface(X, P, Z, cmap="RdYlGn", lw=0.5, rstride=1, cstride=1)
ax.set_axis_off()

#Let's visualize the state in mode 1
state = result.state
fig2 = plt.figure()
X = np.linspace(-5, 5, 100)
P = np.linspace(-5, 5, 100)
Z = state.wigner(1, X, P)
X, P = np.meshgrid(X, P)
ax = fig2.add_subplot(111, projection="3d")
fig2.set_size_inches(4.8, 5)
ax.plot_surface(X, P, Z, cmap="RdYlGn", lw=0.5, rstride=1, cstride=1)
ax.set_axis_off()

In [ ]:


def Visualize_Fock_State(nVal):
    #Let's define the Strawberry Fields program, we will only use a single mode
    prog = sf.Program(1)
    #Let's define the Strawberry Fields engine, we will use the Gaussian backend
    eng = sf.Engine('fock', backend_options={'cutoff_dim': nVal+1})

    #The circuit will place a coherent state with complex amplitude alpha into mode 0, then measure homodyne
    with prog.context as q:
        sf.ops.Fock(nVal)| q[0]

    #Let's run the circuit
    result = eng.run(prog)
    #let's return the samples

    #Let's Plot the Wigner Function of the Fock State
    x = np.linspace(-5, 5, 100) #This creates a 1D array of 100 points between -5 and 5
    y = result.state.wigner(0, x, x) #Returns pre-calculated Wigner function from the state of the quantum register
    X, Y = np.meshgrid(x, x) #Plotting
    plt.contourf(X, Y, y)
    plt.show()

    state = result.state
    fig = plt.figure()
    X = np.linspace(-5, 5, 100)
    P = np.linspace(-5, 5, 100)
    Z = state.wigner(0, X, P)
    X, P = np.meshgrid(X, P)
    ax = fig.add_subplot(111, projection="3d")
    fig.set_size_inches(4.8, 5)
    ax.plot_surface(X, P, Z, cmap="RdYlGn", lw=0.5, rstride=1, cstride=1)
    ax.set_axis_off()

    return

In [ ]:
Visualize_Fock_State(4)

In [ ]:
#SECTION 2 STARTS HERE

In [ ]:
#Define Mutual Information Function
def mutual_information(p_y_given_x, p_x):
    p_y = np.sum(p_y_given_x * p_x[:, np.newaxis], axis=0)
    p_x_y = p_y_given_x * p_x[:, np.newaxis]
    log_p_x_y = np.log2(p_x_y, where=(p_x_y != 0))
    log_p_x = np.log2(p_x, where=(p_x != 0))
    log_p_y = np.log2(p_y, where=(p_y != 0))
    mi = np.sum(p_x_y * (log_p_x_y - log_p_x[:, np.newaxis] - log_p_y))
    return mi

#Define Binary Entropy Function
def binEnt(pVal):
    h2Val= -pVal*np.log2(pVal)-(1-pVal)*np.log2(1-pVal)
    return h2Val

In [ ]:
#Example calculating TPM for Homodyne

#We will use our earlier homodyne sampling function to build a TPM

sample_count=10000

TPM=np.zeros((2,2))

alpha=1

for x in range(2):
    alpha_val=alpha*(-1)**x
    for i in range(sample_count):
        sample=BPSK_Sample_Homodyne(alpha_val)[0]
        if sample>0:
            TPM[x,0]=TPM[x,0]+1
        else:
            TPM[x,1]=TPM[x,1]+1

#Normalize the TPM
for x in range(2):
    TPM[x,:]=TPM[x,:]/sample_count

print(TPM)

#Let's calculate the mutual information, assumping equal priors
p_x=np.array([0.5,0.5])
p_y_given_x=TPM
mi=mutual_information(p_y_given_x, p_x)
print(mi)


In [ ]:
#Lets plot the functional capacities (in the form of PIE) of a few different BPSK receivers

steps=25
mean_arr=np.logspace(-4,1,steps)
cap_arr=np.zeros((steps,12))
hom_arr=1-binEnt(erfc(np.sqrt(2*mean_arr))/2)
dol_arr=1-binEnt((1-np.sqrt(1-np.exp(-4*mean_arr)))/2)
hol_arr=binEnt((1-np.exp(-2*mean_arr))/2)
het_arr=1-binEnt(erfc(np.sqrt(mean_arr))/2)

for k in range(10):
    mVal=2**(k+1)
    for n in range(steps):
        cap_arr[n,k]=(1-np.exp(-mVal*mean_arr[n]))*np.log2(mVal)/(mVal-1)
    mStr="Reduced GM"+str(mVal)
    plt.plot(mean_arr,cap_arr[:,k]/mean_arr[:],label=mStr)

plt.plot(mean_arr,dol_arr/mean_arr,'r--',label='Dolinar')
plt.plot(mean_arr,hom_arr/mean_arr,'b--',label="Homodyne")
plt.plot(mean_arr,het_arr/mean_arr,'g--',label='Heterodyne')
plt.plot(mean_arr,hol_arr/mean_arr,'black',label='BPSK Holevo')
plt.title("Photon Information Efficiency vs Mean Photon Number, BPSK + Receiver")
plt.xlabel("Mean Photon Number (N)")
plt.ylabel("PIE (C(N)/N)")
plt.xscale('log')
plt.grid()
plt.legend()